In [ ]:
import cv
import numpy as np

In [3]:
# 读取待拼接图像
MIN = 10
img1 = cv.imread('1.jpg')
img2 = cv.imread('2.jpg')
# 统一图像大小
height1 = int(img1.shape[0])
width1 = int(img1.shape[1])
dim = (width1, height1)
img2 = cv.resize(img2, dim, interpolation=cv.INTER_AREA)

NameError: name 'cv2' is not defined

In [ ]:
# 创建SIFT特征点检测
sift = cv.SIFT_create()
 
# 检测兴趣点并计算描述子
kp1, describe1 = sift.detectAndCompute(img1, None)
kp2, describe2 = sift.detectAndCompute(img2, None)

In [ ]:
# 使用OpenCV中的FLANN匹配算法进行特征匹配，并返回最近邻和次近邻匹配的结果
FLANN_INDEX_KDTREE = 1
indexParams = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
searchParams = dict(checks=50)
flann = cv.FlannBasedMatcher(indexParams,searchParams)
matches = flann.knnMatch(describe1, describe2, k=2)


In [ ]:
# 储存特征匹配最好的优质匹配点对
'''基于距离阈值选择优质匹配点对，如果最近邻m的距离小于0.65倍的次近邻n的距离，
则认为这个匹配点对是优质的，将它存储在good列表中。'''
good = []
for m,n in matches:
    if m.distance < 0.65 * n.distance:
        good.append(m)
 


In [ ]:
# 可视化特征匹配结果,并保存
pic3 = cv.drawMatches(img1=img1, keypoints1=kp1, img2=img2, keypoints2=kp2, matches1to2=good, outImg=None)
cv.imwrite('pic3.jpg', pic3)

In [ ]:
# RANSAC算法计算单应性矩阵

src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
tge_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
M, mask = cv.findHomography(src_pts, tge_pts, cv.RANSAC, 2)


In [ ]:
# 源图像img2图像扭曲（透视变换）
warpimg = cv.warpPerspective(img2, np.linalg.inv(M), (img1.shape[1] + img2.shape[1], img2.shape[0]))
cv.imwrite('warpimg1.jpg',warpimg)

In [ ]:
# 拼接图像,去除黑边
direct = warpimg.copy()
direct[0:img1.shape[0], 0:img1.shape[1]] = img1
rows, cols = img1.shape[:2]
gray = cv.cvtColor(direct, cv.COLOR_BGR2GRAY)
_, thresh = cv.threshold(gray, 1, 255, cv.THRESH_BINARY)
contours, _ = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
x, y, w, h = cv.boundingRect(contours[0])
direct = direct[y:y+h, x:x+w]

cv.imwrite('result.jpg',direct)